In [95]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [96]:
def constant32(k):
    return tf.constant(k,dtype = tf.float32)

In [97]:
@tf.function
def tf_cov_cubic_f(r,a = constant32(5)):
    c_o = a**2/14/3
    return c_o*(1.-7.*(r/a)**2.+35./4.*(r/a)**3.-7./2.*(r/a)**5.+3./4.*(r/a)**7.)

@tf.function
def tf_cov_cubic_d1_f(r,a = constant32(5)):
    c_o = a**2/14/3
    return (-7* (a - r)**3 *r* (8* a**2 + 9 *a* r + 3* r**2)* (c_o))/(4* a**7)

@tf.function
def tf_cov_cubic_d2_f(r,a = constant32(5)):
    c_o = a**2/14/3
    return (-7 * (4.* a**5. - 15. *a**4. * r + 20. *( a**2)*(r**3) - 9* r**5) * 
            (c_o))/(2*a**7)


In [98]:
@tf.function
def K_Z(x1,x2):
    r = squared_euclidean_distance(x1,x2)
    return tf_cov_cubic_f(r)

# cross-cov space-grad
@tf.function
def K_ZGx(x1,x2):
    r = squared_euclidean_distance(x1,x2)
    hx = x2[0] - x1[0]
    return -hx/r * tf_cov_cubic_d1_f(r)

@tf.function
def K_ZGy(x1,x2):
    r = squared_euclidean_distance(x1,x2)
    hy = x2[1] - x1[1]   
    return -hy/r * tf_cov_cubic_d1_f(r)

# cov grad
@tf.function
def K_Gx(hx,hy):
    r = tf.sqrt(hx,hy)
    if r == 0:
        return 1/3.
    else:
        return (hx**2/r**3 - 1/r) * tf_cov_cubic_d1_f(r) - (hx/r)**2 * tf_cov_cubic_d2_f(r)

@tf.function
def K_Gy(x1,x2):
    r = squared_euclidean_distance(x1,x2)
    hy = x2[1] - x1[1]
    if r == 0:
        return 1/3.
    else:
        return (hy**2/r**3 - 1/r) * tf_cov_cubic_d1_f(r) - (hy/r)**2 * tf_cov_cubic_d2_f(r)

# cross-cov grad
@tf.function
def K_GxGy(x1,x2):
    r = squared_euclidean_distance(x1,x2)
    hx = x2[0] - x1[0]
    hy = x2[1] - x1[1]
    if r == 0:
        return constant32(0)
    else:
        return hx * hy /r**2 * (1/r * tf_cov_cubic_d1_f(r) - tf_cov_cubic_d2_f(r))


In [99]:
x_1 = constant32([[1., 3.],
                  [3., 6.],
                  [5., 9.]])
x_2 = constant32([[1., 3.],
                  [3., 6.],
                  [5., 9.]])

x_1_tiled = tf.tile(x_1,[2,1])
x_2_tiled = tf.tile(x_1,[2,1])

In [100]:
@tf.function
def squared_euclidean_distance(x_1,x_2):
    sqd = tf.sqrt(tf.reshape(tf.reduce_sum(x_1**2,1),shape =(x_1.shape[0],1))+\
    tf.reshape(tf.reduce_sum(x_2**2,1),shape =(1,x_2.shape[0]))-\
    2*tf.tensordot(x_1,tf.transpose(x_2),1))
    return sqd

In [101]:
dist = squared_euclidean_distance(x_1,x_2)

In [102]:
tf.tile(x_1[:,0] - tf.reshape(x_2[:,0],[x_2.shape[0],1]),[1,2])

<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
array([[ 0.,  2.,  4.,  0.,  2.,  4.],
       [-2.,  0.,  2., -2.,  0.,  2.],
       [-4., -2.,  0., -4., -2.,  0.]], dtype=float32)>

In [103]:
#Cartesian distances
def cartesian_dist(x_1,x_2):
    return tf.concat([
        tf.tile(x_1[:,0] - tf.reshape(x_2[:,0],[x_2.shape[0],1]),[1,2]),
        tf.tile(x_1[:,1] - tf.reshape(x_2[:,1],[x_2.shape[0],1]),[1,2])],axis = 0) 
h_u = cartesian_dist(x_1,x_2)

In [104]:
dist_tiled = squared_euclidean_distance(x_1_tiled,x_2_tiled)

In [105]:
h_u

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[ 0.,  2.,  4.,  0.,  2.,  4.],
       [-2.,  0.,  2., -2.,  0.,  2.],
       [-4., -2.,  0., -4., -2.,  0.],
       [ 0.,  3.,  6.,  0.,  3.,  6.],
       [-3.,  0.,  3., -3.,  0.,  3.],
       [-6., -3.,  0., -6., -3.,  0.]], dtype=float32)>

In [106]:
h_v = tf.transpose(h_u)

In [107]:
h_v

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[ 0., -2., -4.,  0., -3., -6.],
       [ 2.,  0., -2.,  3.,  0., -3.],
       [ 4.,  2.,  0.,  6.,  3.,  0.],
       [ 0., -2., -4.,  0., -3., -6.],
       [ 2.,  0., -2.,  3.,  0., -3.],
       [ 4.,  2.,  0.,  6.,  3.,  0.]], dtype=float32)>

In [108]:
tf.matmul(h_u,h_v)/dist_tiled**2

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[        inf,  1.2307696 , -0.1538462 ,         inf,  1.8461543 ,
        -0.23076929],
       [ 1.2307696 ,         inf,  1.2307696 ,  1.8461543 ,         inf,
         1.8461543 ],
       [-0.1538462 ,  1.2307696 ,         inf, -0.23076929,  1.8461543 ,
                inf],
       [        inf,  1.8461543 , -0.23076929,         inf,  2.7692316 ,
        -0.34615394],
       [ 1.8461543 ,         inf,  1.8461543 ,  2.7692316 ,         inf,
         2.7692316 ],
       [-0.23076929,  1.8461543 ,         inf, -0.34615386,  2.7692308 ,
                inf]], dtype=float32)>

In [109]:
a = tf.concat([tf.ones([x_1.shape[0],x_1.shape[0]]),tf.zeros([x_1.shape[0],x_1.shape[0]])],axis = 1)
b = tf.concat([tf.zeros([x_1.shape[0],x_1.shape[0]]),tf.ones([x_1.shape[0],x_1.shape[0]])],axis = 1)

perpendicularity_matrix = tf.concat([a,b],axis = 0)

In [110]:
c_o_T = -1
a_T = 10

In [113]:
index = tf.where(dist_tiled==0).numpy()

In [164]:
C_G*0

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[ 0.,  0.,  0., nan,  0.,  0.],
       [ 0.,  0.,  0.,  0., nan,  0.],
       [ 0.,  0.,  0.,  0.,  0., nan],
       [nan,  0.,  0.,  0.,  0.,  0.],
       [ 0., nan,  0.,  0.,  0.,  0.],
       [ 0.,  0., nan,  0.,  0.,  0.]], dtype=float32)>

In [163]:
C_G*constant32(tf.math.is_nan(C_G).numpy()*-1 +1)

<tf.Tensor: shape=(6, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [149]:
def cov_gradients(dist_tiled):
    C_G = (h_u*h_v / dist_tiled**2)*\
    (-c_o_T*((-14/a_T**2)+105/4*dist_tiled/a_T**3 - 35/2 * dist_tiled**3 / a_T **5 + 21 /4 * dist_tiled**5/a_T**7)+
        c_o_T * 7 * (9 * dist_tiled ** 5 - 20 * a_T ** 2 * dist_tiled ** 3 +
                                       15 * a_T ** 4 * dist_tiled - 4 * a_T ** 5) / (2 * a_T ** 7)-
        perpendicularity_matrix * c_o_T * ((-14 / a_T ** 2) + 105 / 4 * dist_tiled / a_T ** 3 -
                                   35 / 2 * dist_tiled ** 3 / a_T ** 5 +
                                   21 / 4 * dist_tiled ** 5 / a_T ** 7)) +\
        1/3*tf.eye(dist_tiled.shape[0]) ## adding nugget effect
    return C_G

In [93]:
dist_tiled = dist_tiled + tf.eye(dist_tiled.shape[0])

In [37]:
C_G = cov_gradients(dist_tiled)

### Test if it works properly


In [21]:
x = constant32([[0,1]])

In [22]:
d_T = squared_euclidean_distance(x,x)
d_T = d_T + tf.ones(d_T.shape)

In [23]:
h_u = cartesian_dist(x,x)

In [24]:
h_v = tf.transpose(h_u)

In [ ]:
a = tf.concat([tf.ones([x.shape[0],x.shape[0]]),tf.zeros([x.shape[0],x.shape[0]])],axis = 1)
b = tf.concat([tf.zeros([x.shape[0],x.shape[0]]),tf.ones([x.shape[0],x.shape[0]])],axis = 1)

perpendicularity_matrix = tf.concat([a,b],axis = 0)

In [ ]:
cov_gradients(d_T)

In [ ]:
cov_gradients(x)